<a href="https://colab.research.google.com/github/juancuevas-ops/api-nasa/blob/master/Copy_of_Pyspark_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark en Google Colab Automatico

1. Instalacion Marzo/2020
2. Instalacion Java
3. Instalacion de Spark

## Instalacion Marzo/ 2020
De forma General para usar pyspark en Colab Marzo/2020 seria con los siguientes comandos en una celda en Colab:
```python
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os # libreria de manejo del sistema operativo
os.system("wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz")
os.system("tar xf /spark-2.4.5-bin-hadoop2.7.tgz")
# instalar pyspark
!pip install -q pyspark
```

```python
# Variables de Entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{ver_spark}-bin-hadoop2.7"
```
```python
# Cargar Pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()
spark
```
Pero cuadno salga una nueva version de spark sera necesario actualizar los
links de descarga, ya qeu siempre borran las versiones 2.x.x cuando sale una nueva.

Lo mejor es configurar automaticamente para que descargue la version que sea
mayor que 2.3.4 que es la anterior y menor que spark 3.0.0 que aun se encuentra en desarrollo

Para esto el siguiente codigo detecta la version actual de spark, la descarga, la descomprime y luego realiza la instalacion de spark en google colab.


## Instalacion de Java
Google Colaboratory funciona en un ambiente linux, por lo tanto se pueden usar comandos shell de linux antecedidos del caracter '!'

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

## Instalacion de Spark

Obtener automaticamente la ultima version de spark de 

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
#Obtener las versiones de spark la pagina web
url = 'https://downloads.apache.org/spark/' 
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)

In [ ]:
# leer la pagina web y obtener las versiones de spark disponibles
link_files = []
for link in soup.find_all('a'):
  link_files.append(link.get('href'))
spark_link = [x for x in link_files if 'spark' in x]  
print(spark_link)

['spark-2.3.4/', 'spark-2.4.5/', 'spark-3.0.0-preview2/']


La version a usar seran las superiores a spark-2.3.4  y menores a spark-3.0.0

obtener la version y eliminar el caracter '/' del final

In [ ]:
ver_spark = spark_link[1][:-1] # obtener la version y eliminar el caracter '/' del final
print(ver_spark)

spark-2.4.5


In [ ]:
import os # libreria de manejo del sistema operativo
#instalar automaticamente la version deseadda de spark
os.system(f"wget -q https://www-us.apache.org/dist/spark/{ver_spark}/{ver_spark}-bin-hadoop2.7.tgz")
os.system(f"tar xf {ver_spark}-bin-hadoop2.7.tgz")
# instalar pyspark
!pip install -q pyspark

     |████████████████████████████████| 217.8MB 63kB/s 
     |████████████████████████████████| 204kB 53.8MB/s 


## Definir variables de entorno

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{ver_spark}-bin-hadoop2.7"

# Cargar pyspark en el sistema

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test_spark").master("local[2]").getOrCreate()
spark

     |████████████████████████████████| 212.3MB 78kB/s 
     |████████████████████████████████| 204kB 20.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=813d1abf5f5cd19b7cc34f93e5661f6b7c0f823729b9549468bd34af54cbd2a4
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


# Ejemplo de Uso de pyspark

Leer archivo de prueba

In [ ]:
archivo = '/content/ventas_prediccion.csv'
df_spark = spark.read.csv(archivo, inferSchema=True, header=True)

# imprimir tipo de archivo
print(type(df_spark))

<class 'pyspark.sql.dataframe.DataFrame'>


¿Numero de registros en el dataframe?

In [ ]:
df_spark.count()

603

Estructura del dataframe

In [ ]:
df_spark.printSchema()

root
 |-- 2017-01-02: string (nullable = true)
 |-- 236: integer (nullable = true)



¿Nombre de las Columnas de dataframe?

In [ ]:
df_spark.columns

['2017-01-02', '236']

Ver los primeros 20 registros del dataframe

In [ ]:
df_spark.show()

+----------+---+
|2017-01-02|236|
+----------+---+
|2017-01-03|237|
|2017-01-04|290|
|2017-01-05|221|
|2017-01-07|128|
|2017-01-09|293|
|2017-01-10|260|
|2017-01-11|204|
|2017-01-12|157|
|2017-01-13|180|
|2017-01-14|147|
|2017-01-16|228|
|2017-01-17|239|
|2017-01-18|193|
|2017-01-19|197|
|2017-01-20|189|
|2017-01-21|153|
|2017-01-22| 58|
|2017-01-23|193|
|2017-01-24|232|
|2017-01-25|206|
+----------+---+
only showing top 20 rows



## Descricipcion Estadistica del dataframe

In [ ]:
df_spark.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
2017-01-02,603,None,None,2017-01-03,2018-11-30
236,603,215.90215588723052,75.10815266449659,51,591


In [ ]:
import pandas as pd

df_spark.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
2017-01-02,603,None,None,2017-01-03,2018-11-30
236,603,215.90215588723052,75.10815266449659,51,591


In [ ]:
df_spark.show()

+----------+---+
|2017-01-02|236|
+----------+---+
|2017-01-03|237|
|2017-01-04|290|
|2017-01-05|221|
|2017-01-07|128|
|2017-01-09|293|
|2017-01-10|260|
|2017-01-11|204|
|2017-01-12|157|
|2017-01-13|180|
|2017-01-14|147|
|2017-01-16|228|
|2017-01-17|239|
|2017-01-18|193|
|2017-01-19|197|
|2017-01-20|189|
|2017-01-21|153|
|2017-01-22| 58|
|2017-01-23|193|
|2017-01-24|232|
|2017-01-25|206|
+----------+---+
only showing top 20 rows



In [ ]:
archivo = '/content/ventas_prediccion.csv'
df_spark = spark.read.csv(archivo, inferSchema=True, header=True)

# imprimir tipo de archivo
print(type(df_spark))

<class 'pyspark.sql.dataframe.DataFrame'>


Descripcion estadistica de una sola columna ('median_house_value')

De esta forma se puede instalar automaticamente spark en google colab y hacer uno de el de forma gratis.

En la version gratis solo se cuenta con una CPU si se quiere aumentar la capacidad de procesamiento es necesario pagar.